In [0]:
!unzip "/content/drive/My Drive/Deep Learning Data/Skin Cancer/train.zip"
!unzip "/content/drive/My Drive/Deep Learning Data/Skin Cancer/ISIC-2017_Training_Part1_GroundTruth.zip"

In [0]:
import os

#List directory
mask_dir = "/content/ISIC-2017_Training_Part1_GroundTruth"
melanoma_dir ="/content/train/melanoma"
nevus_dir = "/content/train/nevus"
keratosis_dir = "/content/train/seborrheic_keratosis"

In [0]:
mask_ids = os.listdir(mask_dir)
melanoma_ids = os.listdir(melanoma_dir)
nevus_mask =  os.listdir(nevus_dir)
keratosis_mask = os.listdir(keratosis_dir)

In [14]:
print(f" Total numer of masks:  {len(mask_ids)}")
print(f" Total numer of melanoma :  {len(melanoma_ids)}")
print(f" Total numer of nevus :  {len(nevus_mask)}")
print(f" Total numer of keratosis :  {len(keratosis_mask)}")

 Total numer of masks:  2000
 Total numer of melanoma :  374
 Total numer of nevus :  1372
 Total numer of keratosis :  254


In [0]:
import torch
import torch.nn as nn
from torch.nn import Sequential

def resblock(in_put, n_in, n_out):
    input_shape = in_put.shape[2]
    resblock = Sequential(nn.Conv2d(in_channels = n_in, out_channels = n_out, kernel_size = 3, stride = 1, padding =1),
                          nn.ReLU(),
                          nn.Conv2d(in_channels = n_out, out_channels = n_out, kernel_size = 1, stride = 1 , padding = 0),
                          nn.ReLU(),
                          nn.Conv2d(in_channels = n_out, out_channels = n_out, kernel_size = 3, stride = 1, padding = 1))
    return resblock(in_put)

In [5]:
### Test block input (64, 3,256,256) - > out_put (64, 64, 256, 256)
image = torch.randint(low = 0, high = 256, size = (64, 3, 256, 256)).type(torch.float32)

out_image = resblock(image, n_in = image.shape[1], n_out = 64)

print(f"Shape of the output image: {out_image.shape}")

Shape of the output image: torch.Size([64, 64, 256, 256])


In [0]:
import torch
import torch.nn as nn
from torch.nn import Sequential, functional as F
class ResUnet(nn.Module):
    def __init__(self, identity_block, image_size, dropout):
        super().__init__()

        self.image_size = image_size
        self.identity = resblock
        self.dropout = nn.Dropout(p = dropout)
        self.maxpool = nn.MaxPool2d(kernel_size =2, stride = 2) ## reduce input size by half


    def forward(self, image):
        ## Image input size as tensor of shape (batch_size, no_channels, image_size, image_size)
        x = F.relu(self.identity(image, n_in = image.shape[1], n_out = 64))
        x = self.dropout(x)
        x1 = F.relu(self.identity(x, n_in = 64, n_out = 64)) 

        ##Maxpooling to reduce size of images
        x2 = self.maxpool(x1)## return tensor (batch_size, 64, image_size/2, image_size/2)
        x2 = F.relu(self.identity(x2, n_in = x2.shape[1], n_out = x2.shape[1]*2))
        x2 = self.dropout(x2)
        x2 = F.relu(self.identity(x2, n_in =x2.shape[1], n_out = x2.shape[1]))## Return (batch_size, 128, image_size/2, image_size/2)

        ### Maxpooling to reduce size of images
        x3 = self.maxpool(x2)
        x3 = F.relu(self.identity(x3, n_in = x3.shape[1], n_out = x3.shape[1]*2))
        x3 = self.dropout(x3)
        x3 = F.relu(self.identity(x3, n_in = x3.shape[1], n_out = x3.shape[1]))

        ##Maxpooling to reduce size of images
        x4= self.maxpool(x3)
        x4 = F.relu(self.identity(x4, n_in = x4.shape[1], n_out = x4.shape[1]*2))
        x4 = self.dropout(x4)
        x4 = F.relu(self.identity(x4, n_in = x4.shape[1], n_out = x4.shape[1]))

        ## Max pooling to reduce size of image
        x5 = self.maxpool(x4)
        x5 = F.relu(self.identity(x5, n_in =x5.shape[1], n_out = x5.shape[1]))
        x5 = self.dropout(x5)
        

        ##Upsampling 
        up1 = nn.ConvTranspose2d(in_channels = x5.shape[1], out_channels = x5.shape[1],kernel_size= 2, stride = 2, padding = 0)(x5)
        cat1 = torch.cat([x4, up1], dim =1)
        cat1 = F.relu(nn.BatchNorm2d(num_features=cat1.shape[1])(cat1))
        cat1 = self.dropout(cat1)  ### return images with 1024 channels

        ## Reduce no_of channel by half
        cat1 = F.relu(self.identity(cat1, n_in = cat1.shape[1], n_out = int(cat1.shape[1]/2)))# 512
        cat1 = self.dropout(cat1)
        cat1 = F.relu(self.identity(cat1, n_in = cat1.shape[1], n_out = int(cat1.shape[1]/2))) #256

        ## Upsampling
        up2 = nn.ConvTranspose2d(in_channels = cat1.shape[1], out_channels = cat1.shape[1],kernel_size= 2, stride = 2, padding = 0)(cat1)
        cat2 = torch.cat([x3, up2], dim =1)# 512
        cat2 = F.relu(nn.BatchNorm2d(num_features=cat2.shape[1])(cat2))
        cat2 = self.dropout(cat2) # 

        ## Reduce no_of channel by half
        cat2 = F.relu(self.identity(cat2, n_in = cat2.shape[1], n_out = int(cat2.shape[1]/2))) #256
        cat2 = self.dropout(cat2)
        cat2 = F.relu(self.identity(cat2, n_in = cat2.shape[1], n_out = int(cat2.shape[1]/2))) #128

        ## Upsampling
        up3 = nn.ConvTranspose2d(in_channels = cat2.shape[1], out_channels = cat2.shape[1],kernel_size= 2, stride = 2, padding = 0)(cat2)
        cat3 = torch.cat([x2, up3], dim =1) #256
        cat3 = F.relu(nn.BatchNorm2d(num_features=cat3.shape[1])(cat3))
        cat3 = self.dropout(cat3)

        ## Reduce no_kf channels by half

        cat3 = F.relu(self.identity(cat3, n_in = cat3.shape[1], n_out = int(cat3.shape[1]/2))) #128
        cat3 = self.dropout(cat3)
        cat3 = F.relu(self.identity(cat3, n_in = cat3.shape[1], n_out = int(cat3.shape[1]/2))) # 64

        ## Upsampling
        up4 = nn.ConvTranspose2d(in_channels = cat3.shape[1], out_channels = cat3.shape[1],kernel_size= 2, stride = 2, padding = 0)(cat3)
        cat4 = torch.cat([x1, up4], dim = 1) #128
        cat4 = F.relu(nn.BatchNorm2d(num_features=cat4.shape[1])(cat4))
        cat4 = self.dropout(cat4)

        ##reduce no_channels by half
        cat4 = F.relu(self.identity(cat4, n_in = cat4.shape[1], n_out = int(cat4.shape[1]/2)))
        cat4 = self.dropout(cat4)
        cat4 = F.relu(self.identity(cat4, n_in = cat4.shape[1], n_out = 1))

        output = torch.sigmoid(cat4)

        return output
        

model = ResUnet(resblock, 256, 0.2)

In [14]:
import torch
## testblock
test_tensor = torch.randint(low = 1, high = 256, size = (16, 3, 256, 256)).type(torch.float32)
with torch.no_grad():
    output = model(test_tensor)

print(f"With input shape of [16, 3, 256, 256]\nExpected output shape is [16, 1, 256, 256]\nActual output shape is {output.shape}")



With input shape of [16, 3, 256, 256]
Expected output shape is [16, 1, 256, 256]
Actual output shape is torch.Size([16, 1, 256, 256])


In [0]:
## Evaluation metrics
def dice_score(y_pred, y_true, eps = 0.001):
    

    pred = y_pred.view(y_pred.shape[0], -1).float()
    true = y_true.view(y_true.shape[0], -1).float()

    numerator = 2 * torch.sum(pred*true, dim = 1) + eps 
    denominator = torch.sum(pred, dim =1 ) + torch.sum(true, dim =1) + eps
    dice_score = numerator /denominator
    return dice_score.unsqueeze(1)

In [0]:

## Customized loss function for image segmentation
def dice_loss(y_pred, y_true, eps = 0.001):
    

    pred = y_pred.view(-1)
    true = y_true.view(-1)
    intersection = (pred * true).sum()
    
    return 1 - ((2. * intersection + eps) /
              (pred.sum() + true.sum() + eps))